In [1]:
"""
Chapter 10. Network Models. Transportation.
Anderson, David R., Dennis J. Sweeney,
Thomas A. Williams,
Jeffrey D. Camm y Kipp Martin
Quantitative Methods for Business, 11a. Ed
"""
import pulp as pl
from itertools import product

In [2]:
sources = {
    'Cleveland': 5_000, 
    'Bedford': 6_000,
    'York': 2_500,
}

In [3]:
sinks = {
    'Boston': 6_000,
    'Chicago': 4_000,
    'St. Louis': 2_000,
    'Lexington': 1_500,
}

In [4]:
costs =  [
    [3, 2, 7, 6],
    [7, 5, 2, 3],
    [2, 5, 4, 5],
]


costs = pl.makeDict((sources, sinks), costs, 0)

In [5]:
# problem
problem = pl.LpProblem('Transportation', pl.LpMinimize)

# variables
vars = pl.LpVariable.dicts('Route', (sources, sinks), 0, None, pl.LpContinuous)
# print('\n', vars)

In [6]:
# objective function
problem += pl.lpSum(costs[source][sink] * vars[source][sink] for source in sources for sink in sinks), 'Sum_Transporting_Costs'

In [7]:

# Supply Constraints
for source, supply in sources.items():
    problem += pl.lpSum(vars[source][sink] for sink in sinks ) <= supply, f'num_products_out_{source}'

# Demand Constraint
for sink, demand in sinks.items():
    problem += pl.lpSum(vars[source][sink] for source in sources) == demand, f'num_products_into_{sink}'

In [8]:
solver = pl.get_solver('GUROBI') 
problem.solve(solver)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-11-19
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x44076c53
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+03]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8500000e+04   3.500000e+03   0.000000e+00      0s
       2    3.9500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.950000000e+04
Gurobi status= 2


1

In [10]:

# The status of the solution is printed to the screen
print("Status:", pl.LpStatus[problem.status])


# Each of the variables is printed with it's resolved optimum value
for v in problem.variables():
    if v.varValue > 1e-10:
        print(f'{v.name}: {v.varValue:,.2f}')


Status: Optimal
Route_Bedford_Chicago: 2,500.00
Route_Bedford_Lexington: 1,500.00
Route_Bedford_St._Louis: 2,000.00
Route_Cleveland_Boston: 3,500.00
Route_Cleveland_Chicago: 1,500.00
Route_York_Boston: 2,500.00


In [11]:

# The optimised objective function value is printed to the screen
print(f'Total Cost of Transportation = {pl.value(problem.objective):,.2f}')

Total Cost of Transportation = 39,500.00
